In [2]:
# iMPORT PACKAGES
import pandas as pd
import datetime as dt
import numpy as np
import math
from datetime import timedelta
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv("out_without_nan_mood_target.csv")
data['date'] = pd.to_datetime(data['date'])
df = data.dropna(subset = ['mood_mean','mood_mean_TARGET'] )

In [5]:
# Create test and train set
def createTrainTest(length_train):
    length_train = length_train
    index_test = np.random.choice(np.arange(len(df)), length_train, False)

    test = df.iloc[index_test]
    train = df.drop(df.index[index_test])
    
    return train, test

In [6]:
# Calculate MSQ
def calculateAccMSQ(solutions, name_target,name_predicted):
    correct = 0
    squared_error = 0

    for index, row in solutions.iterrows():
        squared_error += math.pow(row[name_target]-row[name_predicted],2)
        if (row[name_target] == row[name_predicted]):
            correct = correct + 1
        
    accuracy = correct/ len(solutions.index)
    msq = squared_error / len(solutions.index)
    return accuracy, msq


In [12]:
# Predict row
def predictMoodPersonARIMA(row, order):    
    #get train data
    train_data_person = df.loc[(df['id'] == row.id) & (df['date'] < row.date)]
    train_data_person_correct = train_data_person[['mood_mean_TARGET']]
    
    prediction_mood_target = None
    #fit the model
    if len(train_data_person_correct) > 10:
        model = ARIMA(train_data_person_correct, order=order)
        model_fit = model.fit()
    
        # make prediction on validation
        prediction = model_fit.forecast()
        prediction_mood_target = prediction[0][0]
    
    return prediction_mood_target 

In [13]:
# Find for every row in test set the predicted mood
def predictARIMA(test, order):
    solutions = pd.DataFrame(data={'id': [], 'date': [], 'mood_mean_target': [], 'predicted_mood_mean_target':[]})
    for index, row in test.iterrows():
        prediction_mood_target = predictMoodPersonARIMA(row, order)
    
        #add solution to solution df
        if prediction_mood_target is not None:
            solutions = solutions.append({'id': row.id, 'date': row.date, 'mood_mean_target': row.mood_mean_TARGET,
                                      'predicted_mood_mean_target': prediction_mood_target}, ignore_index=True)
    return solutions

In [14]:
test = pd.read_csv("test_set.csv")
train = pd.read_csv("train_set.csv")

In [15]:
#train, test = createTrainTest(100)

In [16]:
solutions = predictARIMA(test, (0,1,1))
accuracy, msq = calculateAccMSQ(solutions, "mood_mean_target", "predicted_mood_mean_target")
print(msq)

0.4909540987694328


In [11]:
(1,0,0) = 0.23438219609414976
(0,1,0) = 0.43916991521125376
(0,2,0) = 1.591560337768915
(1,2,0) = 0.8812095702372237
(2,2,0) = 0.7547504645377935
(3,2,0) = error
(3,1,0) = 0.32514441411836276
(4,1,0) = error
(1,1,0) = 0.4145036325619722
(3,0,0) = 0.3425771089769928
(4,0,0) = 0.351796666069525
(0,0,1) = 0.3009717375417968
(0,0,2) = 0.3740895788190101
(0,0,3) = error
(1,0,1) = error
(0,1,1) = 0.24253054075758185
(1,1,1) = error
(0,2,1) = 0.5064616756505296

SyntaxError: can't assign to literal (<ipython-input-11-f4155a6eed94>, line 1)

In [17]:
for p in range(4):
    for d in range(4):
        for q in range(4):
            try:
                solutions = predictARIMA(test, (p,d,q))
                accuracy, msq = calculateAccMSQ(solutions, "mood_mean_target", "predicted_mood_mean_target")
                x1= p,d,q
                print (x1,msq)
            except:
                pass

(0, 0, 0) 0.41810922512615606
(0, 0, 1) 0.41641966863411967
(0, 1, 0) 0.49571671193075895
(0, 1, 1) 0.4909540987694328
(0, 2, 0) 1.4683218653686347
(0, 2, 1) 0.5720363783103162
(1, 0, 0) 0.37527300490579213
(1, 1, 0) 0.4702238678638187
(1, 2, 0) 0.8778453673204697
(2, 1, 0) 0.5038950893250652
(3, 0, 0) 0.41875901327480214
